# 📡 XAI Semantic Communication - Training sur Colab

Ce notebook entraîne le modèle JSCC sur COCO (118k images) avec A100.

**Prérequis :**
- Dataset COCO dans `/content/drive/MyDrive/CocoData/train2017/`
- Runtime GPU A100 activé

## 1. Configuration de l'environnement

In [ ]:
# Vérifier le GPU
!nvidia-smi

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Vérifier le dataset COCO
import os
coco_path = '/content/drive/MyDrive/CocoData/train2017'
if os.path.exists(coco_path):
    n_images = len([f for f in os.listdir(coco_path) if f.endswith('.jpg')])
    print(f"✅ Dataset COCO trouvé : {n_images} images")
else:
    print("❌ Dataset non trouvé ! Exécutez la cellule de téléchargement ci-dessous.")

## 2. (Optionnel) Télécharger COCO si pas encore fait

In [ ]:
# Décommenter pour télécharger COCO (~18 Go, 15-20 min)
# !wget -q --show-progress http://images.cocodataset.org/zips/train2017.zip -O /content/train2017.zip
# !unzip -q /content/train2017.zip -d /content/drive/MyDrive/CocoData/
# !rm /content/train2017.zip

## 3. Cloner le projet depuis GitHub

In [ ]:
import os

# Chemin du projet sur Drive (persistant)
DRIVE_PROJECT = '/content/drive/MyDrive/semantic_comm'

# Si le projet n'existe pas encore sur Drive, le cloner depuis GitHub
if not os.path.exists(DRIVE_PROJECT):
    print("📥 Clonage du projet depuis GitHub...")
    !git clone https://github.com/halidou02/Compression-d-image-avec-AE-et-XAI.git {DRIVE_PROJECT}
    print("✅ Projet cloné sur Drive !")
else:
    print("✅ Projet déjà présent sur Drive")
    # Optionnel : mettre à jour depuis GitHub
    # !cd {DRIVE_PROJECT} && git pull

In [ ]:
# Aller dans le dossier du projet
%cd {DRIVE_PROJECT}
!ls -la

In [ ]:
# Installer les dépendances
!pip install -q torch torchvision tqdm pillow

## 4. Configuration des chemins

In [ ]:
import os

# Créer les dossiers de sauvegarde
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('results/noskip', exist_ok=True)

print("✅ Dossiers créés")

In [ ]:
# Patch du chemin dataset dans train_noskip.py
train_file = 'src/train/train_noskip.py'

with open(train_file, 'r') as f:
    content = f.read()

# Remplacer le chemin du dataset
old_path = "root_dir=str(project_root.parent / 'CocoData')"
new_path = "root_dir='/content/drive/MyDrive/CocoData/train2017'"

if old_path in content:
    content = content.replace(old_path, new_path)
    with open(train_file, 'w') as f:
        f.write(content)
    print("✅ Chemin dataset mis à jour")
else:
    print("ℹ️ Chemin déjà configuré ou différent")

## 5. Lancer l'entraînement

In [ ]:
# Entraînement avec batch_size=32 (A100 peut gérer plus)
!python -m src.train.train_noskip --batch_size 32 --epochs 100 --lr 2e-4

## 6. Reprendre l'entraînement (si interrompu)

In [ ]:
# Si la session a été interrompue, relancer avec --resume
!python -m src.train.train_noskip --batch_size 32 --epochs 100 --lr 2e-4 --resume

## 7. Visualiser les métriques

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Charger les métriques
df = pd.read_csv('results/noskip/metrics.csv')
print(df.tail(10))

# Graphiques
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(df['epoch'], df['grid_avg_psnr'])
axes[0, 0].set_title('Grid PSNR')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('dB')

axes[0, 1].plot(df['epoch'], df['grid_avg_ssim'])
axes[0, 1].set_title('Grid SSIM')
axes[0, 1].set_xlabel('Epoch')

axes[1, 0].plot(df['epoch'], df['train_budget'])
axes[1, 0].set_title('Budget Loss')
axes[1, 0].set_xlabel('Epoch')

axes[1, 1].plot(df['epoch'], df['mono_score'])
axes[1, 1].set_title('Mono Score')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylim([0, 1.1])

plt.tight_layout()
plt.savefig('training_curves.png')
plt.show()

## 8. Vérifier les checkpoints

In [ ]:
!ls -lh checkpoints/
print("\n✅ Tous les fichiers sont sur Drive et persistants !")